# Location Analysis for Cafe in Munich

## 1. Introduction and Data description

__Introduction/Business Problem:__ My friends are going to open a new Cafe in Munich. I would like to do some research and recommend them the best place for their purpose based on number of coffee shops in different districts of Munich.

__Data:__ In order to make my decision I am going to use Foursquare API to get a location data of Munich and its districts and search for coffee shop in these districts. The next step is calculating the number of coffee shops per district (that would be my decision’s variable). Districts with many coffee shops are good places for a new one, because there seem to be a lot of customers.

## 2. Methodology

In [3]:
!pip install geopy
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
from pandas.io.html import read_html
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Libraries imported.')

Libraries imported.


### Downloding and Exploring Munich data

In [34]:
wiki_url='https://de.wikipedia.org/wiki/M%C3%BCnchen'
wikitable = read_html(wiki_url,  attrs={"class":"wikitable sortable zebra"})
wikitable[0]
wikitable[0].head()

,Nr.,Stadtbezirk,Fläche(km²),Einwohner,Dichte(Einw./km²),Ausländer(%)
0,1.0,Altstadt-Lehel,315,21.100,6.708,261
1,2.0,Ludwigsvorstadt-Isarvorstadt,440,51.644,11.734,284
2,3.0,Maxvorstadt,430,51.402,11.960,254
3,4.0,Schwabing-West,436,68.527,15.706,227
4,5.0,Au-Haidhausen,422,61.356,14.541,235


In [57]:
# Function uses geopy library to get the latitude and longitude values 
def get_geo(address):
    geolocator = Nominatim(user_agent="muc_explorer", timeout=3)
    location = geolocator.geocode("Muenchen "+address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    return latitude, longitude

In [58]:
lat=[]
lon=[]
for data in wikitable[0]['Stadtbezirk']:
    stadtbezirk_latlon = get_geo(data)
    stadtbezirk_lat = stadtbezirk_latlon[0]
    stadtbezirk_lon = stadtbezirk_latlon[1]
    lat.append(stadtbezirk_lat)
    lon.append(stadtbezirk_lon)

The geograpical coordinate of Altstadt-Lehel are 48.1396562, 11.5879211.
The geograpical coordinate of Ludwigsvorstadt-Isarvorstadt are 48.1303398, 11.5733658.
The geograpical coordinate of Maxvorstadt are 48.1465704, 11.5714445.
The geograpical coordinate of Schwabing-West are 48.164417, 11.5703639.
The geograpical coordinate of Au-Haidhausen are 48.1287531, 11.5905362.
The geograpical coordinate of Sendling are 48.1245878, 11.5128173.
The geograpical coordinate of Sendling-Westpark are 48.1245878, 11.5128173.
The geograpical coordinate of Schwanthalerhöhe are 48.1337822, 11.5410566.
The geograpical coordinate of Neuhausen-Nymphenburg are 48.1571237, 11.5097031.
The geograpical coordinate of Moosach are 48.1801657, 11.5061222.
The geograpical coordinate of Milbertshofen-Am Hart are 48.1959252, 11.5718148.
The geograpical coordinate of Schwabing-Freimann are 48.1777302, 11.5946593.
The geograpical coordinate of Bogenhausen are 48.1547823, 11.6334838.
The geograpical coordinate of Berg 

In [60]:
wikitable[0]["Latitude"]=lat
wikitable[0]["Longitude"]=lon
muc_data=wikitable[0]
muc_data

,Nr.,Stadtbezirk,Fläche(km²),Einwohner,Dichte(Einw./km²),Ausländer(%),Latitude,Longitude
0,1.0,Altstadt-Lehel,315,21.100,6.708,261,48.139656,11.587921
1,2.0,Ludwigsvorstadt-Isarvorstadt,440,51.644,11.734,284,48.130340,11.573366
2,3.0,Maxvorstadt,430,51.402,11.960,254,48.146570,11.571445
3,4.0,Schwabing-West,436,68.527,15.706,227,48.164417,11.570364
4,5.0,Au-Haidhausen,422,61.356,14.541,235,48.128753,11.590536
5,6.0,Sendling,394,40.983,10.405,269,48.124588,11.512817
6,7.0,Sendling-Westpark,781,59.643,7.632,289,48.124588,11.512817
7,8.0,Schwanthalerhöhe,207,29.743,14.367,335,48.133782,11.541057
8,9.0,Neuhausen-Nymphenburg,1291,98.814,7.651,243,48.157124,11.509703
9,10.0,Moosach,1109,54.223,4.888,315,48.180166,11.506122


In [65]:
# Creating a map of Munich with dastricts.

map_muc = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lon, neighborhood in zip(muc_data['Latitude'], muc_data['Longitude'], muc_data['Stadtbezirk']):
    label = '{}'.format(stadtbezirk_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_muc)  
    
map_muc

### Foursquare data

In [66]:
CLIENT_ID = 'PL23KRBHFOP0S31PP4IP5DMYJEFOFEOG2EJPET1VKRP1QQIK' 
CLIENT_SECRET = 'GUACBBELUNYRLCTLYLQ55JCF5FGZKUPNN4KNJU0CJ4F0UBBD' 
VERSION = '20200112' 

### Exploring the first district in Munich

In [69]:
muc_data.loc[0, 'Stadtbezirk']
get_geo(muc_data.loc[0, 'Stadtbezirk'])

The geograpical coordinate of Altstadt-Lehel are 48.1396562, 11.5879211.


(48.1396562, 11.5879211)

In [166]:
# Top 200 venues in Altstadt-Lehel within a radius of 500 meters
radius=3000
LIMIT=200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    stadtbezirk_lat, 
    stadtbezirk_lon, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1ba0c3b57e88001bdf9782'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Munich',
  'headerFullLocation': 'Munich',
  'headerLocationGranularity': 'city',
  'totalResults': 236,
  'suggestedBounds': {'ne': {'lat': 48.15075952700003,
    'lng': 11.58926911257544},
   'sw': {'lat': 48.09675947299997, 'lng': 11.508524087424561}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1a7577f964a520a1ea23e3',
       'name': 'Goldmarie',
       'location': {'address': 'Schmellerstr. 23',
        'crossStreet': 'Lindwurmstr.',
        'lat': 48.125501,
        'lng': 11.551278,
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.125501,


In [167]:
# Extracting the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']      
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [168]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

100 venues were returned by Foursquare.


,name,categories,lat,lng
0,Goldmarie,German Restaurant,48.125501,11.551278
1,Iveria,Caucasian Restaurant,48.125173,11.549754
2,Fitnessclub Sports & Health,Gym,48.122532,11.544660
3,Bahnwärter Thiel,Nightclub,48.123490,11.555772
4,vesperia,Bistro,48.123992,11.554675


### Exploring other districts in Munich

In [188]:
#Repeating exploring process to all the districs in Munich
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Stadtbezirk', 
                  'Stadtbezirk Latitude', 
                  'Stadtbezirk Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [194]:
muc_venues = getNearbyVenues(names=muc_data['Stadtbezirk'],
                                   latitudes=muc_data['Latitude'],
                                   longitudes=muc_data['Longitude'], radius=3000, LIMIT=200)

Altstadt-Lehel
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Schwabing-West
Au-Haidhausen
Sendling
Sendling-Westpark
Schwanthalerhöhe
Neuhausen-Nymphenburg
Moosach
Milbertshofen-Am Hart
Schwabing-Freimann
Bogenhausen
Berg am Laim
Trudering-Riem
Ramersdorf-Perlach
Obergiesing-Fasangarten
Untergiesing-Harlaching
Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln
Hadern
Pasing-Obermenzing
Aubing-Lochhausen-Langwied
Allach-Untermenzing
Feldmoching-Hasenbergl
Laim
Landeshauptstadt München


In [195]:
print(muc_venues.shape)
muc_venues.head()

(2390, 7)


,Stadtbezirk,Stadtbezirk Latitude,Stadtbezirk Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.139656,11.587921,Patisserie Café Dukatz,48.139375,11.586923,Pastry Shop
1,Altstadt-Lehel,48.139656,11.587921,SEITZ Trattoria,48.141419,11.584902,Trattoria/Osteria
2,Altstadt-Lehel,48.139656,11.587921,Eisbachwelle,48.143342,11.587775,Surf Spot
3,Altstadt-Lehel,48.139656,11.587921,Liebighof im Lehel,48.141640,11.586470,German Restaurant
4,Altstadt-Lehel,48.139656,11.587921,Isar Ufer,48.138992,11.592549,Beach


In [196]:
muc_venues.groupby('Stadtbezirk').count()

,Stadtbezirk Latitude,Stadtbezirk Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Stadtbezirk,,,,,,
Allach-Untermenzing,55,55,55,55,55,55
Altstadt-Lehel,100,100,100,100,100,100
Au-Haidhausen,100,100,100,100,100,100
Aubing-Lochhausen-Langwied,38,38,38,38,38,38
Berg am Laim,100,100,100,100,100,100
Bogenhausen,100,100,100,100,100,100
Feldmoching-Hasenbergl,22,22,22,22,22,22
Hadern,100,100,100,100,100,100
Laim,100,100,100,100,100,100


In [197]:
#Number of unique categories 
print('There are {} uniques categories.'.format(len(muc_venues['Venue Category'].unique())))

There are 225 uniques categories.


### Analyzis of each district

In [198]:
muc_onehot = pd.get_dummies(muc_venues[['Venue Category']], prefix="", prefix_sep="")
muc_onehot['Stadtbezirk'] = muc_venues['Stadtbezirk'] 
fixed_columns = [muc_onehot.columns[-1]] + list(muc_onehot.columns[:-1])
muc_onehot = muc_onehot[fixed_columns]
muc_onehot.head()

,Stadtbezirk,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bavarian Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Bosnian Restaurant,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Campground,Canal,Castle,Caucasian Restaurant,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cretan Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Heliport,Hill,Historic Site,History Museum,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Kebab Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Manti Place,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## 3. Results

In [199]:
# Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
muc_grouped = muc_onehot.groupby('Stadtbezirk').mean().reset_index()
muc_grouped.head()

,Stadtbezirk,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bavarian Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Bosnian Restaurant,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Campground,Canal,Castle,Caucasian Restaurant,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cretan Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Heliport,Hill,Historic Site,History Museum,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Kebab Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Manti Place,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,Opera House,Organic Grocery,Outdoor Sculpture,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Allach-Untermenzing,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.00,0.0,0.036364,0.00,0.0,0.00,0.036364,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.018182,0.000000,0.00,0.0,0.036364,0.00,0.018182,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.054545,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.018182,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.036364,0.0,0.00,0.0,0.072727,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.018182,0.0,0.0,0.018182,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.090909,0.0,0.00,0.018182,0.0,0.0,0.0,0.036364,0.00,0.072727,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.00,0.018182,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.018182,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.

In [201]:
for Stadtbezirk in muc_grouped['Stadtbezirk']:
    temp = muc_grouped[muc_grouped['Stadtbezirk'] == Stadtbezirk].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    max_freq=0
    for index,row in temp.iterrows():
        if row['venue']=='Café':
            freq=row['freq']
            #print(row['venue'], row['freq'])
            if freq>max_freq:
                max_freq=freq
                solution_district=Stadtbezirk

print("The biggest proportion of Cafés among other venues in a district in Munich is {} % in {}.".format(max_freq*100,solution_district))


The biggest proportion of Cafés among other venues in a district in Munich is 4.0 % in Untergiesing-Harlaching.


## 4. Conclusion
I would reccomend to my friends a location  for their new Cafe in district Untergiesing-Harlaching in Munich.